In [1]:
# Bring in past data frame
# This csv will need to be swapped out each update

import pandas as pd

df_who_is_funding_tn_politics = pd.read_csv('72624-tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status.csv')

# Convert date to actual date
df_who_is_funding_tn_politics['date-of-donation'] = pd.to_datetime(df_who_is_funding_tn_politics['date-of-donation'])

In [2]:
df_who_is_funding_tn_politics.dtypes


name-of-recipient-of-donation                                        object
name-of-donor                                                        object
donation-amount                                                       int64
date-of-donation                                             datetime64[ns]
report-year                                                           int64
political-party-of-recipient                                         object
city-of-recipient                                                    object
did-this-company-change-its-name-or-bought-out-since-2009            object
original-name-of-recipient-of-donation                               object
original-name-of-donor                                               object
Are they currently an elected official?                              object
dtype: object

In [3]:
# Creating the total file by candidate, pac seperate

# Group by 'original-name-of-recipient-of-donation' and 'report-year', then sum 'donation-amount'
grouped = df_who_is_funding_tn_politics.groupby(['original-name-of-recipient-of-donation', 'report-year']).agg({
    'donation-amount': 'sum'
}).reset_index()

# Merge back the name-of-recipient-of-donation for reference
merged = pd.merge(grouped, df_who_is_funding_tn_politics[['original-name-of-recipient-of-donation', 'name-of-recipient-of-donation']].drop_duplicates(), on='original-name-of-recipient-of-donation')

# Reorder columns
merged = merged[['original-name-of-recipient-of-donation', 'name-of-recipient-of-donation', 'donation-amount', 'report-year']]

# Calculate total donation amounts for each recipient
totals = merged.groupby(['original-name-of-recipient-of-donation', 'name-of-recipient-of-donation']).agg({
    'donation-amount': 'sum'
}).reset_index()

# Add a 'report-year' column with the value 'Total'
totals['report-year'] = 'Total'

# Append the totals to the original DataFrame
merged = pd.concat([merged, totals], ignore_index=True)

# Search for a specific name in 'name-of-recipient-of-donation'
search_name = 'Sexton, Cam'
total_candidates_by_year_2009_2024 = merged[merged['name-of-recipient-of-donation'].str.contains(search_name, na=False, case=False)]

In [4]:
# Total dataframe called: total_candidates_by_year_2009_2024

total_candidates_by_year_2009_2024 = merged


total_candidates_by_year_2009_2024.dtypes

original-name-of-recipient-of-donation    object
name-of-recipient-of-donation             object
donation-amount                            int64
report-year                               object
dtype: object

In [5]:
# Assuming your DataFrame is already loaded as total_candidates_by_year_2009_2024
df = total_candidates_by_year_2009_2024

# Ensure columns are in correct data types
df['donation-amount'] = df['donation-amount'].astype(int)
df['report-year'] = df['report-year'].astype(str)

# Pivot the DataFrame
pivoted_df = df.pivot_table(
    index=['original-name-of-recipient-of-donation', 'name-of-recipient-of-donation'],
    columns='report-year',
    values='donation-amount',
    aggfunc='sum',
    fill_value=0
)

# Reset the index to make it more readable if needed
pivoted_df.reset_index(inplace=True)

# Create a DataFrame for totals
totals_df = df.groupby('name-of-recipient-of-donation').agg({
    'donation-amount': 'sum'
}).reset_index()

# Add a column for 'original-name-of-recipient-of-donation' with 'total'
totals_df['original-name-of-recipient-of-donation'] = 'total'

# Create a dictionary to hold the total amounts by year
totals_by_year = df.groupby(['name-of-recipient-of-donation', 'report-year'])['donation-amount'].sum().unstack(fill_value=0)

# Add the totals_by_year to totals_df
for year in totals_by_year.columns:
    totals_df[year] = totals_df['name-of-recipient-of-donation'].map(totals_by_year[year].to_dict()).fillna(0).astype(int)

# Ensure the columns are in the correct order to match pivoted_df
totals_df = totals_df[['original-name-of-recipient-of-donation', 'name-of-recipient-of-donation'] + list(pivoted_df.columns[2:])]

# Concatenate the pivoted_df and the totals_df DataFrame
final_df = pd.concat([pivoted_df, totals_df], ignore_index=True)

# Display the resulting DataFrame
total_candidates_by_year_2009_2024 = final_df


In [6]:
total_candidates_by_year_2009_2024.sort_values(by='name-of-recipient-of-donation')

,original-name-of-recipient-of-donation,name-of-recipient-of-donation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
0,"Abernathy, Kimi","Abernathy, Kimi",0,0,0,0,0,0,0,0,0,0,62454,78805,0,0,0,0,141259
1209,total,"Abernathy, Kimi",0,0,0,0,0,0,0,0,0,0,62454,78805,0,0,0,0,141259
1,"Abney, Carol Venea","Abney, Carol Venea",0,0,0,0,0,0,0,0,0,3675,3560,15989,0,0,0,0,23224
1210,total,"Abney, Carol Venea",0,0,0,0,0,0,0,0,0,3675,3560,15989,0,0,0,0,23224
2,"Acuff, James","Acuff, James",0,0,0,0,0,0,0,2900,0,370,0,0,0,0,0,0,3270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,total,"Zachary, Jason",0,0,0,0,0,0,54386,18100,26251,32825,65780,27625,17200,85187,28996,25250,381600
1207,"Zachary, Joey","Zachary, Joey",0,0,0,0,0,0,0,7534,0,0,0,0,0,0,0,0,7534
2339,total,"Zachary, Joey",0,0,0,0,0,0,0,7534,0,0,0,0,0,0,0,0,7534
1208,"Zellers, David","Zellers, David",0,471,0,0,0,0,0,0,0,0,0,0,0,0,0,0,471


In [7]:
# Group by 'name-of-donor', 'name-of-recipient-of-donation', and 'report-year', then sum 'donation-amount'
summary_df = df_who_is_funding_tn_politics.groupby(['name-of-donor', 'name-of-recipient-of-donation', 'report-year']).agg({
    'donation-amount': 'sum'
}).reset_index()

donor_to_candidate_database = summary_df
donor_to_candidate_database.dtypes

name-of-donor                    object
name-of-recipient-of-donation    object
report-year                       int64
donation-amount                   int64
dtype: object

In [8]:
# Cleaning up my data to fit dataframe
# Dataframe containing unique donors to candidates, officials: donor_to_candidate_database

# Ensure columns are in correct data types
donor_to_candidate_database['donation-amount'] = donor_to_candidate_database['donation-amount'].astype(int)
donor_to_candidate_database['report-year'] = donor_to_candidate_database['report-year'].astype(str)

# Pivot the DataFrame
pivoted_donor_to_candidate_database = donor_to_candidate_database.pivot_table(
    index=['name-of-donor', 'name-of-recipient-of-donation'],
    columns='report-year',
    values='donation-amount',
    aggfunc='sum',
    fill_value=0
)

# Reset the index to make it more readable if needed
pivoted_donor_to_candidate_database.reset_index(inplace=True)

# Identify columns corresponding to the years 2009 to 2024
year_columns = [str(year) for year in range(2009, 2025)]

# Create a total column by summing only the year columns
pivoted_donor_to_candidate_database['Total'] = pivoted_donor_to_candidate_database[year_columns].sum(axis=1)

# Display the resulting DataFrame
donor_to_candidate_database = pivoted_donor_to_candidate_database



In [9]:
df_everyone_who_ran_for_office = pd.read_csv('df_tn_candidates_09to24.csv')

df_everyone_who_ran_for_office.dtypes 

Candidate name                                         object
Current official                                       object
Party                                                  object
City                                                   object
Election results and every time they ran for office    object
dtype: object

In [10]:
df_everyone_who_ran_for_office

,Candidate name,Current official,Party,City,Election results and every time they ran for office
0,"Abernathy, Kimi",No,Democrat,Nashville,2020 : Senate District 20 - Lost
1,"Abney, Carol Venea",No,Democrat,Celina,2020 : House of Representatives District 38 - ...
2,"Acuff, James",No,Republican,Washburn,2018 : House of Representatives District 35 - ...
3,"Acuff, Robert (Bob)",No,Republican,Elizabthton,2020 : House of Representatives District 4 - Lost
4,"Adams, Sharon ""Layne""",No,Democrat,Chattanooga,2018 : House of Representatives District 39 - ...
...,...,...,...,...,...
1519,"Young, Patti Larimer",No,Democrat,Maryville,2020 : Senate District 2 - Lost
1520,"Younglove-Miner, Sheila",No,Democrat,Mcminnville,2020 : Senate District 16 - Lost
1521,"Zachary, Jason",Yes,Republican,Knoxville,2024 : House of Representatives 14 - TBD | 202...
1522,"Zachary, Joey",No,Republican,Oneida,2016 : House of Representatives District 38 - ...


In [11]:
df_everyone_who_ran_for_office.to_csv('candidates.csv')
total_candidates_by_year_2009_2024.to_csv('donation_totals.csv')
donor_to_candidate_database.to_csv('donations_by_donor.csv')


In [12]:
# Convert each DataFrame to JSON and save as files
df_everyone_who_ran_for_office.to_json('candidates.json', orient='records')
total_candidates_by_year_2009_2024.to_json('donation_totals.json', orient='records')
donor_to_candidate_database.to_json('donations_by_donor.json', orient='records')


In [13]:
df_everyone_who_ran_for_office.dtypes

Candidate name                                         object
Current official                                       object
Party                                                  object
City                                                   object
Election results and every time they ran for office    object
dtype: object

In [14]:
donor_to_candidate_database

report-year,name-of-donor,name-of-recipient-of-donation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
0,"0, 0","Wamp, Zach",1100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1100
1,"0Menifee, James","Coonrod, Demetrus",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,100
2,1 Hour Bonding,"Dixie, Vincent",0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,100
3,10Jin Llc,"Lee, Bill",0,0,0,0,0,0,0,0,0,0,4200,0,0,0,0,0,4200
4,10Th Street Holdings,"Haslam, Bill",0,2500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254929,"Zwiep, Kyle","Madeira, Elizabeth",0,0,0,0,0,0,0,0,0,0,0,550,0,0,0,0,550
254930,"Zwikl, Helen Danielle","Pearson, Justin",0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,25
254931,"Zyga, Ken","Haslam, Bill",0,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500
254932,"Zylman, Janet","Phillips, Allie",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1000,1000
